In [83]:
from scipy import stats
%matplotlib widget
from scipy.io import loadmat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
import matplotlib.cm as cm
from Funciones_auxiliares import Mat_to_dataframe,see_teams,get_cluster_indexs,plot_branch,plot_teams,find_mix
from scipy import cluster

import scipy as sp
import math

### Importamos la data

In [84]:
plt.close('all')
plt.clf()
plt.close()
data = pd.read_pickle('Datos/PCCIBNMZc')
data['Mean'] = data.Bulk.apply(lambda row: np.mean(row,axis = 0))
data['Mean'] = (data['Mean'] - data.Mean.apply(lambda row:np.mean(row)))/data.Mean.apply(lambda row:np.std(row))
del data['Bulk']
Initial = len(data)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Separamos en Test y Train por sessiones

In [85]:
Sessions_list = data.PatientExperiment.unique()
select_random_sessions = np.random.choice(Sessions_list,(10,6), replace = False)
Dfs = {}
for i,sessions in enumerate(select_random_sessions):
    Dfs[i] = data[data.PatientExperiment.isin(sessions)]
for key,values in Dfs.items():
    print(len(values))

In [86]:
len(Train) + len(Test)

1577

Calculamos la matriz de correlacion

## Ahora entrenamos

In [87]:
df_train = Train.copy()

Calculo de la matriz de correlacion

In [88]:
Noise = df_train[df_train.bNoise == 1]
Neuron = df_train[df_train.bNoise == 0]
MU = df_train[df_train.bNoise == 2]

df_train = pd.concat([Neuron,MU,Noise], ignore_index= True, sort = False)
Mean = pd.DataFrame(df_train.Mean.tolist()).T

Corr = Mean.corr()
Corr = Corr.values
t = df_train[df_train.bNoise == 1].index[0]
print(len(df_train),'elements are in the df')
print(len(df_train[df_train.bNoise != 1]),'elements are neurons.',len(df_train[df_train.bNoise == 1]),'elements are noises')

861 elements are in the df
287 elements are neurons. 574 elements are noises


In [89]:
threshold2 = 0.99# For Pearson is defined as 1 - correlation

Separamos en correlaciones entre ruidos y correlaciones mezcladas

In [90]:
Noise_corr = Corr[t:,t:]
Neu_noi_corr = Corr[t:,:t]
Noise_correlated = np.where(Noise_corr>threshold2)
Neuron_noise_correlated = np.where(Neu_noi_corr > threshold2)

In [91]:
plt.figure()
plt.title('Noise correlation')
plt.imshow(Noise_corr)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Cuantas neuronas fueron capturadas por los ruidos?

In [93]:
Neurons_captured = set(Neuron_noise_correlated[1])
print(len(Neurons_captured),'neurons were captured by the noise-database')
print('this represents',len(Neurons_captured)/t*100,'% of neurons' )

94 neurons were captured by the noise-database
this represents 32.752613240418114 % of neurons


Cuantas son los ruidos responsables?

In [94]:
bad_noises = set(Neuron_noise_correlated[0] + t)
print(len(bad_noises), 'noises are responsible for the captured neurons')


62 noises are responsible for the captured neurons


Tiramos estos ruidos

In [95]:
New_df = df_train.drop(bad_noises)
print( 'Is',all(df_train.loc[bad_noises].bNoise == 1),' that all the elements droped are noise')
New_df

Is True  that all the elements droped are noise


,PatientExperiment,Channel,Cluster,ISI,bNoise,Mean,ZeroCrossings
0,HEC017\017context2\017context2_prescr,113,2,"[0, 3, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, ...",0,"[0.4148400945586795, 0.41008625177590113, 0.39...",2
1,HEC017\017context2\017context2_prescr,113,3,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",0,"[0.1465658286378676, 0.13310457368256473, 0.12...",6
2,HEC017\017context2\017context2_prescr,118,3,"[0, 4, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",0,"[0.39807454790263525, 0.3830459993330923, 0.37...",2
3,HEC017\017context2\017context2_prescr,120,3,"[0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...",0,"[0.08402676015159298, 0.05090180060164797, 0.0...",8
4,HEC017\017context2\017context2_prescr,122,2,"[0, 1, 7, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[0.43204238087330993, 0.4127649121444305, 0.39...",2
...,...,...,...,...,...,...,...
856,HEC030\HEC030_story4,107,2,"[0, 23, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",1,"[-0.9319573075572473, -0.6380342844822416, -0....",6
857,HEC030\HEC030_story4,95,3,"[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...",1,"[0.6329774241252774, 0.8106414791428904, 0.966...",3
858,HEC030\HEC030_story4,98,2,"[0, 5, 6, 11, 2, 8, 10, 10, 15, 6, 19, 15, 15,...",1,"[0.5850305135481051, 0.5912792052355617, 0.594...",2
859,HEC032\HEC032_rsvpscr16,81,2,"[0, 0, 0, 36, 1, 0, 2, 4, 0, 0, 1, 0, 0, 1, 0,...",1,"[-0.2555185424972891, -0.3367829648401097, -0....",1


Repetimos todo el proceso y verificamos que no hayan neuronas encontradas

In [96]:
New_df.reset_index(drop = True)
t = New_df[New_df.bNoise == 1].index[0]
print(len(New_df))
Mean= pd.DataFrame(np.array(New_df.Mean.to_list()))
Corr = Mean.T.corr()
Corr = Corr.values
Noise_corr = Corr[t:,t:]
Neu_noi_corr = Corr[t:,:t]
Noise_correlated = np.where(Noise_corr>threshold2)
Neuron_noise_correlated = np.where(Neu_noi_corr > threshold2)
Neuron_noise_correlated
Neurons_captured = set(Neuron_noise_correlated[1])
print(len(Neurons_captured),'neurons were captured')

799
0 neurons were captured


## Entrenamos

In [71]:
Noise_old = New_df[New_df.bNoise == 1]
t2 = len(Noise_old)

Agrego solamente las neuronas.

In [73]:
Noise = Test[Test.bNoise == 1]
Noise.reset_index(inplace = True)
Neuron = Test[Test.bNoise == 0]
Neuron.reset_index(inplace = True)
Mu = Test[Test.bNoise == 2]
Mu.reset_index(inplace = True)

df = pd.concat([Neuron,Mu,Noise_old],ignore_index = True,sort = False)
t = df[df.bNoise == 1].index[0]
print(len(df),'elements are in the df')
print(len(df[df.bNoise != 1]),'elements are neurons.',len(df[df.bNoise == 1]),'elements are noises')

685 elements are in the df
312 elements are neurons. 373 elements are noises


In [74]:
Mean= pd.DataFrame(np.array(df.Mean.to_list()))
Corr = Mean.T.corr()
Corr = Corr.values

In [75]:
Noise_corr = Corr[t:,t:]
Neu_noi_corr = Corr[t:,:t]
Noise_correlated = np.where(Noise_corr>threshold2)
Neuron_noise_correlated = np.where(Neu_noi_corr > threshold2)

In [76]:
Neurons_captured = set(Neuron_noise_correlated[1])
print(len(Neurons_captured),'neurons were captured by the noise-database')
print('this represents',len(Neurons_captured)/(len(df[df.bNoise != 1]))*100,'% of Neurons')

16 neurons were captured by the noise-database
this represents 5.128205128205128 % of Neurons


In [77]:
bad_noises = list(set(Neuron_noise_correlated[0] + t))
len(bad_noises)

14

Cuanto nuevo ruido capturo?

In [78]:
All_noise = pd.concat([Noise_old,Noise],ignore_index=True,sort = False)
t2 = len(Noise_old)
Mean = pd.DataFrame(np.array(All_noise.Mean.to_list()))
Noise_corr = Mean.T.corr().values
old_new_corr = Noise_corr[t2:,:t2]
noise_found = np.where(old_new_corr>threshold2)[0]+t2
noise_not_found = set(np.arange(len(Noise))+t2) - set(noise_found)
print(len(set(noise_found))/len(Noise)*100,'% of noise is found (',len(set(noise_found)),'noises)')
print(len(noise_not_found)/len(Noise)*100,'% of noise is not found (',len(noise_not_found),'noises)')

23.89240506329114 % of noise is found ( 151 noises)
76.10759493670885 % of noise is not found ( 481 noises)


1317